In [ ]:
%pylab inline

In [ ]:
import hydrafloods as hf
import datetime
import ee
from pprint import pprint

ee.Initialize()

In [ ]:
geom = ee.Geometry.Rectangle([103.0,10.5,106.0,13.0])

In [ ]:
date = "2019-02-11"
dt = datetime.datetime.strptime(date,"%Y-%m-%d")
print(dt)

In [ ]:
previous = (dt + datetime.timedelta(0)).strftime('%Y-%m-%d')
tomorrow = (dt + datetime.timedelta(1)).strftime('%Y-%m-%d')

In [ ]:
targetAsset = "users/kelmarkert/hydra_research/"

hand = ee.Image("projects/servir-mekong/SWMT/HAND_MERIT")

## Exporting a Sentinel-1 water map

In [ ]:
# product = "landsat8"

In [ ]:
product = "sentinel1"
worker = hf.processing.Sentinel1(geom,date,tomorrow,collectionid='COPERNICUS/S1_GRD')
waterImage = worker.waterMap(date,qualityBand="VV",initThresh=-14,reverse=True).And(hand.lt(30))

In [ ]:
waterImage = waterImage.rename('water')\
    .set({'system:time_start':ee.Date(date).millis(),'sensor':product})
assetTarget = targetAsset + '{0}_bmaxOtsu_{1}'.format(product,date.replace('-',''))


In [ ]:
# set product to export
description = '{0}_water_{1}'.format(product,date)
hf.geeutils.exportImage(waterImage,geom,assetTarget,description=description,scale=30)


#### Exporting a Landsat8 water map

In [ ]:
product = "landsat8"
worker = hf.processing.Landsat(geom,date,tomorrow,collectionid='LANDSAT/LC08/C01/T1_SR')
waterImage = worker.waterMap(date,qualityBand="mndwi").And(hand.lt(30))

In [ ]:
waterImage = waterImage.rename('water')\
    .set({'system:time_start':ee.Date(date).millis(),'sensor':product})
assetTarget = targetAsset + '{0}_bmaxOtsu_{1}'.format(product,date.replace('-',''))


In [ ]:
# set product to export
description = '{0}_water_{1}'.format(product,date)
hf.geeutils.exportImage(waterImage,geom,assetTarget,description=description,scale=30)